In [1]:
#엔진은 항상필요하다 
from sqlalchemy import create_engine

In [2]:
engine = create_engine("sqlite://", echo = True)

In [3]:
#metadata 같은것
from sqlalchemy.ext.declarative import declarative_base

In [4]:
Base = declarative_base() #metadata 와 비슷한 역할을 하는 베이스를 만든다

## 아까는 메타데이터를 상속받아서 테이블 지금은 베이스를 만들어서 클래스를 만든다.

In [5]:
from sqlalchemy import Column,Integer,String,ForeignKey

In [6]:
class Artist(Base):
    __tablename__ = "artist"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    def __repr__(self):
        return "<T'artist(name='%s')>" % (self.name)
    
class Album(Base):
    __tablename__ = "album"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    artist_id = Column(Integer, ForeignKey("artist.id"))

    
class Genre(Base):
    __tablename__ = "genre"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
   
    
    
class Track(Base):
    __tablename__ = "track"
    
    id = Column(Integer, primary_key=True)
    title = Column(String)
    length = Column(String)
    album_id = Column(Integer, ForeignKey("album.id"))
    genre_id = Column(Integer, ForeignKey("genre.id"))



In [7]:
Base.metadata.create_all(engine) # metadata 가 안에있다. 그래서 아까 배운거. 아까 만든 엔진을  binding 해준다
#클래스를 선언해서 만들어준거 밖에없는데 자기가 알아서 테이블을 만든다

2018-07-10 17:32:31,203 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2018-07-10 17:32:31,205 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:32:31,207 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2018-07-10 17:32:31,208 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:32:31,210 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("artist")
2018-07-10 17:32:31,211 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:32:31,212 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("album")
2018-07-10 17:32:31,213 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:32:31,214 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("genre")
2018-07-10 17:32:31,214 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:32:31,215 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("track")
2018-07-10 17:32:31,216 INFO sqlalchemy.engine.base.Engine ()
2018-07-10 17:32:31,218 INFO sqlalchemy.engin

In [8]:
artist1 = Artist(name="Led Zepplin")
artist2 = Artist(name="AC/DC")

#conn 이 없는 대신에 session 이 있다. 
#session 이 필요한 이유를 잘 못들엇음 ㅠ

In [9]:
from sqlalchemy.orm import sessionmaker #session 을 만들어야한다

In [10]:
Session = sessionmaker(bind =engine) #엔진과 비인딩을 해준다

In [11]:
session = Session() #이때 세션을 생성

In [12]:
session.add_all([artist1,artist2])

In [13]:
session.commit() #세션을 통해 바인딩 된 엔진을 통해서 우리가만든 인스턴스 artist1,2, 를 SQL로만들고 데이터베이스에다가 물리적으로 저장하는 과정까지 해준다 

2018-07-10 17:32:37,277 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:32:37,278 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-07-10 17:32:37,279 INFO sqlalchemy.engine.base.Engine ('Led Zepplin',)
2018-07-10 17:32:37,280 INFO sqlalchemy.engine.base.Engine INSERT INTO artist (name) VALUES (?)
2018-07-10 17:32:37,281 INFO sqlalchemy.engine.base.Engine ('AC/DC',)
2018-07-10 17:32:37,282 INFO sqlalchemy.engine.base.Engine COMMIT


session 에는 dirty 라는 게있다. 메모리를 관리한다, 데이터베이스와 싱크와 맞아야한다 , 따라서 계속 감시한다 , 메모리의 값들 이 변경과 , 데이터베이스의  따라서 바뀐게있으면 dirty 에 표시가되고 , commit 을 해야한다는것을 사용자 에게 알려준다 . 

In [14]:
session.dirty

IdentitySet([])

In [15]:
session.new

IdentitySet([])

In [16]:
artist1.name = "xyz"

In [17]:
session.dirty #바꼈는데 반영되지않는값이 있다고 알려준다

IdentitySet([<T'artist(name='xyz')>])

In [18]:
session.commit() # 따라서 커밋한다

2018-07-10 17:32:39,330 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:32:39,332 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id 
FROM artist 
WHERE artist.id = ?
2018-07-10 17:32:39,333 INFO sqlalchemy.engine.base.Engine (1,)
2018-07-10 17:32:39,334 INFO sqlalchemy.engine.base.Engine UPDATE artist SET name=? WHERE artist.id = ?
2018-07-10 17:32:39,335 INFO sqlalchemy.engine.base.Engine ('xyz', 1)
2018-07-10 17:32:39,336 INFO sqlalchemy.engine.base.Engine COMMIT


In [19]:
session.dirty

IdentitySet([])

In [20]:
session.add_all([
    Album(title="Hysteria",artist_id=1),
    Album(title="21st Century BreakDown",artist_id=2),
    
])

In [21]:
session.dirty

IdentitySet([])

In [22]:
session.new #new 에 있다. 값이 변경된것이아니라 , 작업이 메모리에만 있으니까

IdentitySet([<__main__.Album object at 0x000001E6609734E0>, <__main__.Album object at 0x000001E660973550>])

In [23]:
session.add_all([
    Genre(name="Rock"),
    Genre(name="Metal"),
])


In [24]:
session.add_all([
    Track(title="1-1",album_id=1,genre_id=1),
    Track(title="1-2",album_id=1,genre_id=2),
    Track(title="2-1",album_id=2,genre_id=1),
    Track(title="2-2",album_id=2,genre_id=2),
    
    
])

In [47]:
session.commit()

2018-07-10 17:45:33,476 INFO sqlalchemy.engine.base.Engine INSERT INTO track (title, length, album_id, genre_id) VALUES (?, ?, ?, ?)
2018-07-10 17:45:33,477 INFO sqlalchemy.engine.base.Engine ('5-1', None, 5, 5)
2018-07-10 17:45:33,478 INFO sqlalchemy.engine.base.Engine COMMIT


In [42]:
temp = Track(title="5-1",album_id=5,genre_id=5)

In [43]:
type(temp)

__main__.Track

In [44]:
session.add(temp)

In [48]:
temp.id

2018-07-10 17:45:37,632 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2018-07-10 17:45:37,634 INFO sqlalchemy.engine.base.Engine SELECT track.id AS track_id, track.title AS track_title, track.length AS track_length, track.album_id AS track_album_id, track.genre_id AS track_genre_id 
FROM track 
WHERE track.id = ?
2018-07-10 17:45:37,635 INFO sqlalchemy.engine.base.Engine (6,)


6

In [46]:
session.new

IdentitySet([<__main__.Track object at 0x000001E660A1F6D8>])

In [32]:
for row in session.query(Artist):
    print(row)

2018-07-10 17:34:45,266 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist
2018-07-10 17:34:45,267 INFO sqlalchemy.engine.base.Engine ()
<T'artist(name='xyz')>
<T'artist(name='AC/DC')>


In [34]:
for row in session.query(Artist).filter(Artist.id==1): # 필터를 통해서 where 처럼해서 메모리에 있는것을 불러온다.
    print(row) 

2018-07-10 17:35:08,213 INFO sqlalchemy.engine.base.Engine SELECT artist.id AS artist_id, artist.name AS artist_name 
FROM artist 
WHERE artist.id = ?
2018-07-10 17:35:08,214 INFO sqlalchemy.engine.base.Engine (1,)
<T'artist(name='xyz')>


filter 와  filter_by 가 있는데 해당 테이블에 있느냐 없느냐로 해석됨. 

foreign 키를 설정해서 , 클래스르 161부터 목요일 부터  - 장 부터 한줄이면된다 . 릴레이션쉽을 사용하면  목요일날하겠다 